## The Oracle

In quantum computing, an oracle is a device used to encode information about a function without explicitly revealing its explicit rule, it is also known as black box. It plays a crucial role in many quantum algorithms, such as Grover's search algorithm and the Deutsch-Jozsa algorithm. The oracle can be thought of as a tool that, when given a specific input, produces an output according to an unknown function $f(x)$. 

Great! But how is it possible to construct and design an oracle for a quantum algorithm? In general, an oracle is represented by a unitary operator $U_f$. This operator acts on a quantum state to evaluate a binary function $f(x)$. For example, in the context of Grover's [[3](#grover_paper), [4](#childs_notes)] and Deutsch-Jozsa algorithm [[1](#dj_paper), [2](#dj_tutorial), [4](#childs_notes)], the oracle $U_f$ takes the action $U_f|x\rangle |y\rangle = |x\rangle |y\oplus f(x)\rangle$. The $\oplus$ represents the XOR operation:

* $x \oplus y$ is equals to $0$ if $x=y$;
* $x \oplus y$ is equals to $1$ if $x\neq y$.  


Both of these algorithms provide speedups in the number of steps required for its execution, when compared to its classical counterparts. 

![Oracle_fig](https://docs.classiq.io/resources/Oracle_fig.png)

However, in general, the construction of an oracle is not a trivial task. Implementing a quantum oracle requires encoding the desired function into a quantum algorithm. This often involves designing a series of quantum gates that represent the function's logic. While the specifics can vary, the general approach is to create a model where the function $f(x)$ influences the qubits in a way that reflects its output.

# Classiq Concepts

In this tutorial, we will be using some Classiq elements that you can take a look:

* [Arithmetic operations](https://docs.classiq.io/latest/explore/tutorials/technology_demonstrations/arithmetic_expressions/arithmetic_expressions/)
* [Within-Apply](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/within-apply/ )

You can also take a look at our [Deutsch-Jozsa Tutorial](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/) and [Grover's Algorithm Tutorial](https://docs.classiq.io/latest/explore/built_in_apps/grover/grover/) if you want to see an oracle being applied in a quantum algorithm.

One of the main challenges in using quantum oracles is constructing efficient and scalable representations for complex functions. As quantum computing technology advances, finding optimal methods for oracle construction and integration into quantum algorithms remains an active area of research. For instance, let's examine the oracle for the following function, which can be used in the context of the [Deutsch-Jozsa algorithm](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/), as a balanced function:

![Oracle_table](https://docs.classiq.io/resources/Oracle_table.png)

<details><summary>How do we build the oracle for the function displayed in the table?</summary>

The function $f(x)$ assumes its value as $1$ only when the integer value of $| x \rangle$ is even. Therefore, everytime the last qubit of $| x \rangle$ is $1$, $f(x)=0$ and our oracle should simply be a bit flip on the last qubit followed by a CNOT controlled by the last qubit of $| x \rangle$.

</details>

We can see that the function $f(x)$ follows a rule: Everytime the integer value of the qubit $| x \rangle$ is divisible by $2$, $f$ is $1$. In other words, if we were to build an oracle for this function, it should flip the $y$ bit every time $x = 0$ mod $2$. Even though, in this case, we can construct such oracle in a clever way, it is not always an easy task to build it.

If we have an arithmetic expression for the rule of $f(x)$, it is possible to construct this oracle with a few lines of code in Classiq, letting the building blocks of the quantum circuit being automatically done by the synthesizer, and even being able to optimize it in the depth of the circuit (number of operations) or in the width (number of qubits):

In [1]:
from classiq import *
#This is the construction of our oracle
@qfunc
def oracle_function(res:QNum, x:QNum):
    res ^= x %2 == 0

* The `x %2 == 0` part of the code represents the oddity check we want to do: `x%2` means $x$ mod $2$. 

* The `^=` part represents an XOR operation between its left hand side, and right hand side, assigning its result in the qubit `res`. A short introdution to this concept can be seen [here](https://docs.classiq.io/latest/explore/functions/function_usage_examples/arithmetic/bitwise_xor/bitwise_xor_example/).

* Therefore, `res ^= x %2 == 0` means that we are doing an XOR operation that follows the rule `x %2 == 0`, assigning the result in `res`.

Thats it! Our oracle is constructed. In Classiq's IDE, it would look like this:

![Oracle_gif](https://docs.classiq.io/resources/Oracle_gif.gif)

Now we are able to append it as a subroutine in a quantum circuit. For example, we can apply it in the Deutsch-Jozsa algorithm:

In [5]:
@qfunc
def prepare_minus(out: QNum):
    #Here, we prepare the minus state for the Deutsch-Jozsa algorithm
    X(out)
    H(out)

@qfunc
def main(y: Output[QNum], x: Output[QNum]):    
    allocate(3,x)
    allocate(1,y)
    #Here, we employ the Deutsch-Jozsa algorithm, using the oracle we built previously.
    prepare_minus(y)
    within_apply(lambda: apply_to_all(H,x), lambda: oracle_function(y,x))

qmodel = create_model(main)
qprogram = synthesize(qmodel)
show(qprogram)

Opening: https://platform.classiq.io/circuit/e19b0b0b-cfd8-426e-9cea-63a8997dfd5a?version=0.43.0


In the IDE, the Deutsch-Jozsa algorithm using this oracle would look like this:

![DJ_gif](https://docs.classiq.io/resources/DJ-circuit.gif)

# Results

Now that we have synthesized our program, we can execute it directly through Classiq's IDE. In the Deutsch-Jozsa algorithm, we can see that when we have a balanced function, the measured qubits $|x\rangle$ would never be $|00 \dots 0\rangle$. 

In the following execution with 1000 shots in the simulator shown below, we can see that in this case $|x\rangle$ (the first three qubits) is never $|00 \dots 0\rangle$, as we would expect.


![dj_outputs](https://docs.classiq.io/resources/dj_outputs.png)

# The complete code

You can see the complete Python version of the Deutsch-Jozsa implementation applying this Oracle below:

In [ ]:
from classiq import *
@qfunc
def oracle_function(res:QNum, x:QNum):
    res ^= x %2 == 0

@qfunc
def prepare_minus(out: QNum):
    X(out)
    H(out)

@qfunc
def main(y: Output[QNum], x: Output[QNum]):    
    allocate(3,x)
    allocate(1,y)
    prepare_minus(y)
    within_apply(lambda: apply_to_all(H,x), lambda: oracle_function(y,x))

qmodel = create_model(main)
qprogram = synthesize(qmodel)
show(qprogram)

# References

<a id='dj_paper'>[1]</a>: [Rapid solution of problems by quantum computation (David Deutsch and Richard Jozsa)](https://doi.org/10.1098/rspa.1992.0167)

<a id='dj_tutorial'>[2]</a>: [Deutsch-Jozsa Algorithm Tutorial (Classiq's documentation)](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/)

<a id='grover_paper'>[3]</a>: [A fast quantum mechanical algorithm for database search (Lov K. Grover)](https://doi.org/10.1145/237814.237866)